In [ ]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

Importuojame bibliotekas

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Apsirašome hyperparametrus

In [ ]:
hidden_size = 256
MAX_LENGTH = 30
n_epochs = 5
DROPOUT_P = 0.15
learning_rate = 0.0015
batch_size = 64

Apsirašome pagalbinę klasę

In [ ]:
SOS_token = 0
EOS_token = 1

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1


Pasiverčiame ASCII į Unicode ir sutvarkome tekstą


In [ ]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z!?]+", r" ", s)
    return s.strip()

Nusiskaitome kalbų failus.

In [ ]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs



def readLangs_TwoFiles(lang1, lang2, reverse=False):
    print("Reading lines...")

    with open('/content/%s.txt' % (lang1), encoding='utf-8') as f:
        lines_lang1 = [normalizeString(s) for s in f.read().strip().split("\n")[:-1]]
    with open('/content/%s.txt' % (lang2), encoding='utf-8') as f:
        lines_lang2 = [normalizeString(s) for s in f.read().strip().split("\n")[:-1]]

    pairs = list(zip(lines_lang1, lines_lang2))

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

Išfiltruojame iki max length

In [ ]:


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

Atliekame visą procesą

In [ ]:
def prepareData(lang1, lang2, reverse=False, two_files=False):
    if two_files:
      input_lang, output_lang, pairs = readLangs_TwoFiles(lang1, lang2, reverse)
    else:
      input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs
lang1 = "eng"
lang2 = "spa"
two_files = False
reverse = False
# input_lang, output_lang, pairs = prepareData(lang1, lang2, False)
input_lang, output_lang, pairs = prepareData(lang1, lang2, reverse, two_files)
print(random.choice(pairs))

Reading lines...
Read 118964 sentence pairs
Trimmed to 118944 sentence pairs
Counting words...
Counted words:
eng 12920
spa 24771
['she s been wearing the same hat for a month', 'ella ha estado usando el mismo sombrero por un mes']


Seq2Seq Encoder modelis

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input):
        embedded = self.dropout(self.embedding(input))
        output, hidden = self.gru(embedded)
        return output, hidden

Decoder modelis

In [ ]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden  = self.forward_step(decoder_input, decoder_hidden)
            decoder_outputs.append(decoder_output)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        return decoder_outputs, decoder_hidden, None # We return `None` for consistency in the training loop

    def forward_step(self, input, hidden):
        output = self.embedding(input)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.out(output)
        return output, hidden

Attention mechanizmas

In [ ]:
class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor=None):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(MAX_LENGTH):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

Pasiruošiame duomenis treniravimui

In [ ]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(1, -1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

def get_dataloader(batch_size, lang1, lang2, test_split=0.2):
    input_lang, output_lang, pairs = prepareData(lang1, lang2, reverse, two_files)

    # Shuffle the pairs
    random.shuffle(pairs)

    # Split into training and testing sets
    test_size = int(len(pairs) * test_split)
    train_pairs = pairs[:-test_size]
    test_pairs = pairs[-test_size:]

    print(f"Number of training pairs: {len(train_pairs)}")
    print(f"Number of testing pairs: {len(test_pairs)}")

    # Create training dataloader
    n_train = len(train_pairs)
    input_ids_train = np.zeros((n_train, MAX_LENGTH), dtype=np.int32)
    target_ids_train = np.zeros((n_train, MAX_LENGTH), dtype=np.int32)

    for idx, (inp, tgt) in enumerate(train_pairs):
        inp_ids = indexesFromSentence(input_lang, inp)
        tgt_ids = indexesFromSentence(output_lang, tgt)
        inp_ids.append(EOS_token)
        tgt_ids.append(EOS_token)
        input_ids_train[idx, :len(inp_ids)] = inp_ids
        target_ids_train[idx, :len(tgt_ids)] = tgt_ids

    train_data = TensorDataset(torch.LongTensor(input_ids_train).to(device),
                               torch.LongTensor(target_ids_train).to(device))
    train_sampler = RandomSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

    return input_lang, output_lang, train_dataloader, test_pairs

Treniruojame modelį

In [ ]:
def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:
        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

Pagalbinė funkcija apskaičiuoti laiką

In [ ]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

Apsirašome treniravimo procesą

In [ ]:
def train(train_dataloader, encoder, decoder, n_epochs, learning_rate=0.001,
               print_every=100, plot_every=100):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    for epoch in range(1, n_epochs + 1):
        loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if epoch % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, epoch / n_epochs),
                                        epoch, epoch / n_epochs * 100, print_loss_avg))

    showPlot(plot_losses)

Rezultato gavimas

In [ ]:
def evaluate(encoder, decoder, sentence, input_lang, output_lang):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, decoder_hidden, decoder_attn = decoder(encoder_outputs, encoder_hidden)

        _, topi = decoder_outputs.topk(1)
        decoded_ids = topi.squeeze()

        decoded_words = []
        for idx in decoded_ids:
            if idx.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            decoded_words.append(output_lang.index2word[idx.item()])
    return decoded_words, decoder_attn

Testuojame atsitiktines poras

In [ ]:
def evaluateRandomly_test(encoder, decoder, test_pairs, n=10):
    for i in range(n):
        pair = random.choice(test_pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, _ = evaluate(encoder, decoder, pair[0], input_lang, output_lang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

Atliekamas treniravimas

In [ ]:


input_lang, output_lang, train_dataloader, test_pairs = get_dataloader(batch_size, lang1, lang2)

encoder = EncoderRNN(input_lang.n_words, hidden_size, DROPOUT_P).to(device)
decoder = AttnDecoderRNN(hidden_size, output_lang.n_words, DROPOUT_P).to(device)

train(train_dataloader, encoder, decoder, n_epochs, learning_rate=learning_rate, print_every=1, plot_every=99)

Reading lines...
Read 118964 sentence pairs
Trimmed to 118944 sentence pairs
Counting words...
Counted words:
eng 12920
spa 24771
Number of training pairs: 95156
Number of testing pairs: 23788
2m 15s (- 9m 0s) (1 20%) 1.0250
4m 27s (- 6m 41s) (2 40%) 0.5928
6m 41s (- 4m 27s) (3 60%) 0.4307
8m 55s (- 2m 13s) (4 80%) 0.3432
11m 8s (- 0m 0s) (5 100%) 0.2904


Testuojame su naujomis poromis


In [ ]:
encoder.eval()
decoder.eval()
evaluateRandomly_test(encoder, decoder, test_pairs)

> who did tom bring ?
= a quien trajo tom ?
< tom hizo trampas ? <EOS>

> it s one of those
= es uno de aquellos
< uno de esos <EOS>

> is it true that mary killed her husband ?
= es cierto que mary mato a su esposo ?
< es que mary se burlo de la mujer de mary ? <EOS>

> i can not afford to buy a used car
= no me puedo permitir comprarme un coche de segunda mano
< no puedo comprar un coche de coche <EOS>

> tom gave mary a cup of coffee
= tom le dio a mary una taza de cafe
< mary una taza de cafe <EOS>

> america is the land of opportunity
= estados unidos es la tierra de la oportunidad
< estados unidos es la tierra de estados unidos <EOS>

> i m defenseless
= estoy indefenso
< estoy <EOS>

> the child painted flowers
= el nino pintaba flores
< los bomberos <EOS>

> you must show respect to your guests
= tienes que ser educado con tus invitados
< debes respetar los invitados <EOS>

> it s a waste of time to study when you re sleepy
= es una perdida de tiempo estudiar con sueno
< se hac

Atliekame matavimus

In [ ]:
!pip install -q --upgrade rouge-score
!pip install -q --upgrade keras-hub

In [ ]:
import keras_hub
rouge_1 = keras_hub.metrics.RougeN(order=1)
rouge_2 = keras_hub.metrics.RougeN(order=2)

# Import the BLEU metric
from nltk.translate.bleu_score import sentence_bleu

bleu_scores = [] # Initialize a list to store BLEU scores

for test_pair in test_pairs[:30]:
    input_sentence = test_pair[0]
    reference_sentence = test_pair[1]

    output_words, _ = evaluate(encoder, decoder, input_sentence, input_lang, output_lang)
    translated_sentence = ' '.join(output_words)
    translated_sentence = translated_sentence.replace("SOS", "").replace("<EOS>", "").replace("[END]", "").strip()

    rouge_1(reference_sentence, translated_sentence)
    rouge_2(reference_sentence, translated_sentence)

    # Calculate BLEU score
    # BLEU score requires the reference to be a list of strings
    reference = [reference_sentence.split()]
    candidate = translated_sentence.split()
    bleu_scores.append(sentence_bleu(reference, candidate))

print("ROUGE-1 Score: ", rouge_1.result())
print("ROUGE-2 Score: ", rouge_2.result())

# Print the average BLEU score
print("Average BLEU Score: ", sum(bleu_scores) / len(bleu_scores))

/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

ROUGE-1 Score:  {'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.5395767092704773>, 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.4558902978897095>, 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.48591935634613037>}
ROUGE-2 Score:  {'precision': <tf.Tensor: shape=(), dtype=float32, numpy=0.28158727288246155>, 'recall': <tf.Tensor: shape=(), dtype=float32, numpy=0.20612554252147675>, 'f1_score': <tf.Tensor: shape=(), dtype=float32, numpy=0.23115907609462738>}
Average BLEU Score:  0.03670637522247806
